# Rapport TAL
## Partie Présidents
Membres du binôme :
- KRISNI Almehdi (3800519)
- ARICHANDRA Santhos (3802651)

### Introduction
Dans cette partie, nous allons nous intéresser à la prédiction de l'interlocuteur lors d'un débat présidentiel entre Chirac et Mitterrand. Entre d'autres termes, qui a dit quoi ?

Nous disposons donc d'une base de données d'apprentissage de 57000 lignes représentant un ensemble de phrases, chacune de ces dernières étiquetées par un C (Chirac) ou un M (Mitterrand), désignant la personne l'ayant prononcé. On peut alors labéliser notre ensemble d'apprentissage en 2 classes distinctes. 

Notre objectif est de mettre en place des algorithmes allant nous permettre de labelliser de nouveaux débats entre Chirac et Mitterrand.

### Mise en place des import et des librairies utilisées

In [54]:
import numpy as np
import matplotlib.pyplot as plt

# Pour retirer les messages d'alerte
import warnings
warnings.filterwarnings('ignore')

import codecs
import re
import os.path

import copy

# Librairies pour le traitement de texte
import re
import unicodedata
import string

# Librairie pour l'utilisation de compteurs
from collections import Counter

# Librairies pour le traitement de texte et la mise en place de vecteurs
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Librairies pour la phase de Machine Learning
import numpy as np
import sklearn.naive_bayes as nb
from sklearn import svm
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

# Libraire pour les statistiques (moyenne, ...)
import statistics

# Importation des librairies standards
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

# Importation de votre librairie iads
import sys
sys.path.append('../')

# Iportation de utils
from iads import utils as ut

# commande TRES utile pour recharger automatiquement le code que vous modifiez dans les modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Première analyse des données

On extrait dans un premier les informations à la base de données d'apprentissage. On utilise la fonction d'extraction "load_pres" située dans le fichier utils.py.

On considère 2 classes, la classe -1 représentant Mitterrand et la classe +1 représentant Chirac.

In [55]:
appdata = ut.load_pres("AFDpresidentutf8/corpus.tache1.learn.utf8")
appX = appdata[0]
appY = appdata[1]

On s'intéresse à la répartition des labels dans les données. Une personne parle-t-elle beaucoup plus que l'autre ?

In [56]:
print("Le nombre de lignes prononcées par Chirac :\t", str(appY.count(1)))
print("Le nombre de lignes prononcées par Mitterrand :\t", str(appY.count(-1)), "\n")
print("Proportion de Chirac dans la base d'apprentissage :\t", str((appY.count(1) / len(appY)) * 100), "%")
print("Proportion de Mitterrand dans la base d'apprentissage :\t", str((appY.count(-1) / len(appY)) * 100), "%")

Le nombre de lignes prononcées par Chirac :	 49890
Le nombre de lignes prononcées par Mitterrand :	 7523 

Proportion de Chirac dans la base d'apprentissage :	 86.89669587027329 %
Proportion de Mitterrand dans la base d'apprentissage :	 13.103304129726718 %


On remarque donc qu'il y a presque 87% des lignes dans la base de données d'apprentissage ayant été prononcées par Mitterrand. Il se peut que cette différence puisse jouer un rôle important lors de l'apprentissage.

### Nettoyage des données et analyse avancée

On effectue alors un premier nettoyage des données en supprimant les chiffres, la ponctuation, les accents, les majuscules et les stopwords de la langue française.

La liste des stopwords utilisés est la fusion entre la liste de la libraire nltk.stopwords et spacy.lang.fr.stop_words.

Le stemmer Snowball est tiré de la librairie nltk.stem.snowball.

In [60]:
appX = appdata[0]
appY = appdata[1]

In [61]:
print('\033[1m', "Sans aucune modification, on obtient comme premières lignes :",'\033[0m')
for s in appX[:5] :
    print("\t", s)
print()

test1AppX = ut.transform(appX, punc=True, accentMaj=True, nb=True, stopW=False, stem=False)
print('\033[1m', "Sans suppression des stopwords et sans stemming, on obtient comme premières lignes :",'\033[0m')
for s in test1AppX[:5] :
    print("\t", s)
print()

test2AppX = ut.transform(appX, punc=True, accentMaj=True, nb=True, stopW=True, stem=False)
print('\033[1m', "Avec suppression des stopwords et sans stemming, on obtient comme premières lignes :", '\033[0m')
for s in test2AppX[:5] :
    print("\t", s)
print()

test3AppX = ut.transform(appX, punc=True, accentMaj=True, nb=True, stopW=False, stem=True)
print('\033[1m', "Sans suppresion des stopwords et avec stemming, on obtient comme premières lignes :", '\033[0m')
for s in test3AppX[:5] :
    print("\t", s)
print()

test4AppX = ut.transform(appX, punc=True, accentMaj=True, nb=True, stopW=True, stem=True)
print('\033[1m', "Avec suppression des stopwords et un stemming, on obtient comme premières lignes :",'\033[0m')
for s in test4AppX[:5] :
    print("\t", s)

 Sans aucune modification, on obtient comme premières lignes : 
	  Quand je dis chers amis, il ne s'agit pas là d'une formule diplomatique, mais de l'expression de ce que je ressens.

	  D'abord merci de cet exceptionnel accueil que les Congolais, les Brazavillois, nous ont réservé cet après-midi.

	  C'est toujours très émouvant de venir en Afrique car c'est probablement l'une des rares terres du monde où l'on ait conservé cette convivialité, cette amitié, ce respect de l'autre qui s'expriment avec chaleur, avec spontanéité et qui réchauffent le coeur de ceux qui arrivent et de ceux qui reçoivent.

	  Aucun citoyen français ne peut être indifférent à un séjour à Brazzaville.

	  Le Congo, que naguère le <nom> qualifia de "refuge pour la liberté", de "base de départ pour la libération", de "môle pour la Résistance", comment ne pas être heureux de s'y retrouver ?


 Sans suppression des stopwords et sans stemming, on obtient comme premières lignes : 
	  quand je dis chers amis  il ne s 

On crée un Counter allant mettre en avant les mots les plus souvent utilisés par chacun des Présidents, après nettoyage des données en fonction des paramètres selectionnés.

In [6]:
# Création du dictionnaire et counter de mots de Mitterrand
motsM = [appX[i] for i in range(len(appY)) if appY[i] == -1]
motsM = (' '.join(motsM)).split()
counterM = Counter(motsM)

print("Les 10 mots les plus utilisés par Mitterrand sont :\n", [i[0] for i in counterM.most_common(10)], "\n")

# Creation du dictionnaire et counter de mots de Chirac
motsC = [appX[i] for i in range(len(appY)) if appY[i] == 1]
motsC = (' '.join(motsC)).split()
counterC = Counter(motsC)

print("Les 10 mots les plus utilisés par Chirac sont :\n", [i[0] for i in counterC.most_common(10)])

Les 10 mots les plus utilisés par Mitterrand sont :
 ['de', 'la', 'et', 'à', 'le', 'les', 'que', 'des', 'qui', 'en'] 

Les 10 mots les plus utilisés par Chirac sont :
 ['de', 'la', 'et', 'à', 'les', 'le', 'des', 'que', 'en', 'qui']


On s'intéresse à mettre en place deux dictionnaires exclusifs pour Chirac et Mitterrand. Il s'agit de dictionnaires comportant les mots qu'ils utilisent de manière exclusive.

In [7]:
# Dictionnaire exclusif de Mitterrand
excluM = copy.deepcopy(counterM)
for w in counterC.keys() :
    if w in excluM.keys() :
        del excluM[w]
        
print("Le ratio de mots exclusifs utilisés par Mitterrand est de :\t", len(excluM.keys()) / len(counterM.keys()))
print("Les 30 mots exclusifs les plus utilisés par Mitterrand sont :\n", [i[0] for i in excluM.most_common(30)], "\n")

# Dictionnaire exclusif de Chirac
excluC = copy.deepcopy(counterC)
for w in counterM.keys() :
    if w in excluC.keys() :
        del excluC[w]
        
print("Le ratio de mots exclusifs utilisés par Chirac est de :\t\t", len(excluC.keys()) / len(counterC.keys()))
print("Les 30 mots exclusifs les plus utilisés par Chirac sont :\n", [i[0] for i in excluC.most_common(30)])

Le ratio de mots exclusifs utilisés par Mitterrand est de :	 0.3066925478943371
Les 30 mots exclusifs les plus utilisés par Mitterrand sont :
 ["'", '-plan', '320', '-cadre', 'convenait', '-état', '-nature', '-rapport', 'Moi', "l'-état", '-entreprise', "'Suite", 'demeurant,', '-concours', 'Douze,', 'Bon,', 'répété', '"mais', 'au-cours', "',", '340', 'vient,', 'imaginez', 'Seulement', "'<date>',", 'Nièvre', 'définitions', "l'événement,", '500000', 'direz'] 

Le ratio de mots exclusifs utilisés par Chirac est de :		 0.6868985231062411
Les 30 mots exclusifs les plus utilisés par Chirac sont :
 ['Messieurs,', 'mondialisation', 'Maire,', '".', '*', "l'État", 'Mesdames,', '000', 'À', '",', 'attentes', 'défi', 'lève', 'accueil', 'Madame', 'civilisations', 'constituent', 'Secrétaire', 'Légion', 'respectueux', 'Ministre,', 'Unies', 'accueillir', 'éthique', 'Gouvernement,', "l'euro", 'Affaires', 'XXIe', 'mondialisation.', 'superbe']


On décide de supprimer les mots en communs entre Chirac et Mitterrand parmi les 30 mots les plus utilisés par chacun d'entre eux.

In [8]:
counterM_smc30, counterC_smc30 = ut.suppN_sharedmostcommon(30, copy.copy(counterM), copy.copy(counterC))

print("Les 10 mots les plus utilisés par Mitterrand sont :\n", [i[0] for i in counterM_smc30.most_common(10)], "\n")
print("Les 10 mots les plus utilisés par Chirac sont :\n", [i[0] for i in counterC_smc30.most_common(10)])

Les 10 mots les plus utilisés par Mitterrand sont :
 ['il', 'se', '-', "c'est", 'Je', 'tout', 'sont', 'mais', 'cette', 'bien'] 

Les 10 mots les plus utilisés par Chirac sont :
 ['notre', 'nos', 'avec', 'se', 'sont', 'aux', 'Je', 'aussi', 'cette', 'leur']


On remarque donc une bien meilleure différence entre les deux listes des 10 mots les plus utilisés pour chacun des Présidents, comparé à avant la suppression des mots communs partagés (suppression de 'france', 'europe', 'pays', 'faire' et autres mots).

### Modèles de Machine Learning selectionnés

Nous avons choisi d'utiliser les principaux modèles vus en cours et en TME, c'est-à-dire :
- SVM linéaire (LinearSVC)
- Naive Bayes (MultinomialNB)
- régression logistique (LogisticRegression)

Nous utiliserons donc la libraire sklearn puisqu'elle propose tous ces modèles. On importe tous les composants nécessaires de la librairie et d'autres élements utiles.

In [9]:
# Import des modèles utilisés
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

# Import de librairies pour le temps et la validation
import time
from sklearn.model_selection import cross_validate, cross_val_score, KFold

# Import de libraires pour la mise en forme du texte
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Afin d'utiliser les données, il faut les mettre sous forme vectorielle.

In [10]:
# Utilisation de la libraire CountVectorizer
vectPres = CountVectorizer()
Xpres = vectPres.fit_transform(appX)
Ypres = appY

## Première campagne d'expériences

Durant notre première campagne, nous nous sommes intéressés uniquement à l'utilisation des différents modèles en faisant varier quelques paramètres afin d'obtenir les meilleurs résultats comme l'utilisation ou non de la validation croisée, le nombre d'ensemble et le mélange ou non des données.

### Utilisation du modèle SVM linéaire (LinearSVC)

##### Sans validation croisée

In [234]:
# Création du classifieur
svc = LinearSVC(max_iter=1000)
svc.fit(Xpres, Ypres)

# Liste pour sauvegarder les scores de précision
resultSVM = []

# Score sur les données
res = svc.score(Xpres, Ypres)

# Affichage des informations
print("Score sur les données d'entraînement :", res)
print('Nombre de paramètres vecteur de poids du SVM :', len(svc.coef_[0]),'\nVecteur de poids du SVM :', svc.coef_)

Score sur les données d'entraînement : 0.9763294027485064
Nombre de paramètres vecteur de poids du SVM : 28524 
Vecteur de poids du SVM : [[ 1.55348538 -0.22581442  0.3904539  ... -0.33502112  0.
  -0.42396068]]


##### Validation croisée, KFold à n = 5, sans mélange

In [235]:
# Prédictions en utilisant la validation croisée sans mélange des données d'apprentissage
svc = LinearSVC(max_iter=1000)

# KFold permet de créer des sous-ensemble de données tel que à chaque itération, les sous-ensembles ait le même format
# Il s'agit de l'équivalent de la fonction de validation croisée que l'on trouve dans le fichier iads/utils.py
# https://machinelearningmastery.com/k-fold-cross-validation/#:~:text=Nevertheless%2C%20the%20KFold%20class%20can,with%20very%20large%20data%20samples.
kfold = KFold(n_splits=5, shuffle=False)

# Un fit est effectué lors de l'appel de la fonction cross_val_score
scores_cv = cross_val_score(svc, Xpres, Ypres, cv=kfold)
resultSVM.append(1 - statistics.mean(scores_cv))

print("Score sur les données d'entraînement:", scores_cv, "\nPrécision moyenne = %.5f" % statistics.mean(scores_cv))

Score sur les données d'entraînement: [0.8582252  0.86972046 0.87999652 0.89487894 0.89026302] 
Précision moyenne = 0.87862


##### Validation croisée, KFold à n = 10, sans mélange

In [236]:
# Prédictions en utilisant la validation croisée sans mélange des données d'apprentissage
svc = LinearSVC(max_iter=1000)

kfold = KFold(n_splits=10, shuffle=False)
scores_cv = cross_val_score(svc, Xpres, Ypres, cv=kfold)
resultSVM.append(1 - statistics.mean(scores_cv))

print("Score sur les données d'entraînement:", scores_cv, "\nPrécision moyenne = %.5f" % statistics.mean(scores_cv))

Score sur les données d'entraînement: [0.84343434 0.87513062 0.87008011 0.86918655 0.87998607 0.88434071
 0.90123672 0.89705626 0.89078558 0.89200488] 
Précision moyenne = 0.88032


##### Validation croisée, KFold à n = 5, avec mélange

In [237]:
# Prédictions en utilisant la validation croisée sans mélange des données d'apprentissage
svc = LinearSVC(max_iter=1000)

kfold = KFold(n_splits=5, shuffle=True)
scores_cv = cross_val_score(svc, Xpres, Ypres, cv=kfold)
resultSVM.append(1 - statistics.mean(scores_cv))

print("Score sur les données d'entraînement:", scores_cv, "\nPrécision moyenne = %.5f" % statistics.mean(scores_cv))

Score sur les données d'entraînement: [0.88626666 0.88940172 0.88887921 0.88460199 0.88730186] 
Précision moyenne = 0.88729


##### Validation croisée, KFold à n = 10, avec mélange

In [238]:
# Prédictions en utilisant la validation croisée sans mélange des données d'apprentissage
clf = LinearSVC(max_iter=1000)

kfold = KFold(n_splits=10, shuffle=True)
scores_cv = cross_val_score(clf, Xpres, Ypres, cv=kfold)
resultSVM.append(1 - statistics.mean(scores_cv))

print("Score sur les données d'entraînement:", scores_cv, "\nPrécision moyenne = %.5f" % statistics.mean(scores_cv))

Score sur les données d'entraînement: [0.89184953 0.88627656 0.88749565 0.88730186 0.88852116 0.88730186
 0.88956628 0.89287581 0.89235325 0.88329559] 
Précision moyenne = 0.88868


### Utilisation du modèle Naive Bayes (MultinomialNB)

#### Sans validation croisée

In [239]:
# Création du classifieur
clf = MultinomialNB()
clf.fit(Xpres, Ypres)

# Liste pour sauvegarder les scores de précision
resultNB = []

# Score sur les données
print("Score sur les données d'entraînement :", clf.score(Xpres, Ypres))
print('Nombre de paramètres vecteur de poids du SVM :', len(clf.coef_[0]),'\nVecteur de poids du SVM :', clf.coef_)

Score sur les données d'entraînement : 0.9158030411230906
Nombre de paramètres vecteur de poids du SVM : 28524 
Vecteur de poids du SVM : [[ -8.48585104  -9.69294397 -10.3553195  ... -13.85182706 -13.85182706
  -13.85182706]]


##### Validation croisée, KFold à n = 5, sans mélange

In [240]:
# Prédictions en utilisant la validation croisée sans mélange des données d'apprentissage
clf = MultinomialNB()

kfold = KFold(n_splits=5, shuffle=False)
scores_cv = cross_val_score(clf, Xpres, Ypres, cv=kfold)
resultNB.append(1 - statistics.mean(scores_cv))

print("Score sur les données d'entraînement:", scores_cv, "\nPrécision moyenne = %.5f" % statistics.mean(scores_cv))

Score sur les données d'entraînement: [0.85927023 0.8664983  0.87929983 0.89757882 0.88033444] 
Précision moyenne = 0.87660


##### Validation croisée, KFold à n = 10, sans mélange

In [241]:
# Prédictions en utilisant la validation croisée sans mélange des données d'apprentissage
clf = MultinomialNB()

kfold = KFold(n_splits=10, shuffle=False)
scores_cv = cross_val_score(clf, Xpres, Ypres, cv=kfold)
resultNB.append(1 - statistics.mean(scores_cv))

print("Score sur les données d'entraînement:", scores_cv, "\nPrécision moyenne = %.5f" % statistics.mean(scores_cv))

Score sur les données d'entraînement: [0.84691745 0.87600139 0.87617555 0.86204494 0.88416652 0.87632817
 0.90175928 0.89148232 0.87772165 0.88346978] 
Précision moyenne = 0.87761


##### Validation croisée, KFold à n = 5, avec mélange

In [242]:
# Prédictions en utilisant la validation croisée sans mélange des données d'apprentissage
clf = MultinomialNB()

kfold = KFold(n_splits=5, shuffle=True)
scores_cv = cross_val_score(clf, Xpres, Ypres, cv=kfold)
resultNB.append(1 - statistics.mean(scores_cv))

print("Score sur les données d'entraînement:", scores_cv, "\nPrécision moyenne = %.5f" % statistics.mean(scores_cv))

Score sur les données d'entraînement: [0.88347993 0.88914047 0.88086737 0.89069848 0.88477617] 
Précision moyenne = 0.88579


##### Validation croisée, KFold à n = 10, avec mélange

In [243]:
# Prédictions en utilisant la validation croisée sans mélange des données d'apprentissage
clf = MultinomialNB()

kfold = KFold(n_splits=10, shuffle=True)
scores_cv = cross_val_score(clf, Xpres, Ypres, cv=kfold)
resultNB.append(1 - statistics.mean(scores_cv))

print("Score sur les données d'entraînement:", scores_cv, "\nPrécision moyenne = %.5f" % statistics.mean(scores_cv))

Score sur les données d'entraînement: [0.88174852 0.88557994 0.88749565 0.88712768 0.88904372 0.88312141
 0.88364396 0.88468908 0.88573419 0.88886953] 
Précision moyenne = 0.88571


### Utilisation du modèle de Regression Logistique (LogisticRegression)

#### Sans validation croisée

In [244]:
# Création du classifieur
lin = LogisticRegression(max_iter=1000)
lin.fit(Xpres, Ypres)

# Liste pour sauvegarder les scores de précision
resultLR = []

# Score sur les données
print("Score sur les données d'entraînement :", lin.score(Xpres, Ypres))
print('Nombre de paramètres vecteur de poids du SVM :', len(lin.coef_[0]),'\nVecteur de poids du SVM :', lin.coef_)

Score sur les données d'entraînement : 0.9445247592008779
Nombre de paramètres vecteur de poids du SVM : 28524 
Vecteur de poids du SVM : [[ 2.34084434 -0.52298687  0.13207444 ... -0.37166223 -0.02158416
  -0.22511148]]


##### Validation croisée, KFold à n = 5, sans mélange

In [ ]:
# Prédictions en utilisant la validation croisée sans mélange des données d'apprentissage
lin = LogisticRegression(max_iter=1000)

kfold = KFold(n_splits=5, shuffle=False)
scores_cv = cross_val_score(lin, Xpres, Ypres, cv=kfold)
resultLR.append(1 - statistics.mean(scores_cv))

print("Score sur les données d'entraînement:", scores_cv, "\nPrécision moyenne = %.5f" % statistics.mean(scores_cv))

##### Validation croisée, KFold à n = 10,  sans mélange

In [ ]:
# Prédictions en utilisant la validation croisée sans mélange des données d'apprentissage
lin = LogisticRegression(max_iter=1000)

kfold = KFold(n_splits=10, shuffle=False)
scores_cv = cross_val_score(lin, Xpres, Ypres, cv=kfold)
resultLR.append(1 - statistics.mean(scores_cv))

print("Score sur les données d'entraînement:", scores_cv, "\nPrécision moyenne = %.5f" % statistics.mean(scores_cv))

##### Validation croisée, KFold à n = 5, avec mélange

In [ ]:
# Prédictions en utilisant la validation croisée sans mélange des données d'apprentissage
lin = LogisticRegression(max_iter=1000)

kfold = KFold(n_splits=5, shuffle=True)
scores_cv = cross_val_score(lin, Xpres, Ypres, cv=kfold)
resultLR.append(1 - statistics.mean(scores_cv))

print("Score sur les données d'entraînement:", scores_cv, "\nPrécision moyenne = %.5f" % statistics.mean(scores_cv))

##### Validation croisée, KFold à n = 10, avec mélange

In [ ]:
# Prédictions en utilisant la validation croisée sans mélange des données d'apprentissage
lin = LogisticRegression(max_iter=1000)

kfold = KFold(n_splits=10, shuffle=True)
scores_cv = cross_val_score(lin, Xpres, Ypres, cv=kfold)
resultLR.append(1 - statistics.mean(scores_cv))

print("Score sur les données d'entraînement:", scores_cv, "\nPrécision moyenne = %.5f" % statistics.mean(scores_cv))

### Affichage et comparaisons des données

Maintenant que nous avons récupérer tous les scores de précision moyenne, nous allons désormais les comparer afin de réaliser de premières observations.

In [ ]:
# Regroupement des résultats
xlabels = ["crossval, n=5, no shuffle", "crossval, n=5, shuffle", 
           "crossval, n=10, no shuffle", "crossval, n=10, shuffle"]
x = np.arange(4)

# Création du graphique
plt.figure(figsize=(8, 4))
ax = plt.subplot(111)

# Mise en place des informations
ax.bar(x - 0.1, resultSVM, width=0.1, color='g', align='center', label="LinearSVC")
ax.bar(x, resultNB, width=0.1, color='r', align='center', label="MultinomialNB")
ax.bar(x + 0.1, resultLR, width=0.1, color='b', align='center', label="LogisticRegression")
ax.set_xlabel('All models and settings', fontsize=10)
ax.set_ylabel('Mean Loss', fontsize=10)
plt.title('Mean losses of different models and settings')
plt.xticks(x, xlabels, fontsize=8)

# Mise en place de la légende
plt.legend(loc='center left', bbox_to_anchor=(1, 1))

# Affichage et sauvegarde
plt.savefig('first_exp_cmp.png', bbox_inches='tight')

On remarque que le modèle le plus précis est la Regression Logistique, puisque son taux d'erreur moyen est constamment inférieur ou égal à 10%. On remarque tout de même qu'en augmentant le nombre d'itérations

## Deuxième campagne d'expériences

Nous avons évoqué précedemment le fait que chaque Président utilise plus souvent certains mots que d'autres

## Troisième campagne d'expériences

### Recherche des meilleurs paramètres par GridSearch

On ne peut pas essayer chaque combinaison de paramètres à la main, il faut donc mettre en place un GridSearch avec tous les paramètres utilisés afin de trouver la meilleure.

On crée un dictionnaire allant contenir tous les champs de paramètres et valeurs possibles.

In [ ]:
# Dictionnaire de paramètres globaux
mainParameters = {"processing" : 
                  { "punc": [False, True],
                    "accentMaj": [False, True],
                    "nb": [False, True],
                    "stopW": [False, True],
                    "stem": [False, True] },
                  }

# 

# Création des Vectorizer
# CountVectorizer
vectorizerC_pres = CountVectorizer()
vectorizerC_pres1G = CountVectorizer(ngram_range=(1,1))
vectorizerC_pres2G = CountVectorizer(ngram_range=(2,2))
vectorizerC_pres12G = CountVectorizer(ngram_range=(1,2))
vectorizerC_pres13G = CountVectorizer(ngram_range=(1,3))
vectorizerC_pres3G = CountVectorizer(ngram_range=(3,3))

vectorizerTFIDF_pres = TfidfVectorizer()
vectorizerTFIDF_pres1G = TfidfVectorizer(ngram_range=(1,1))
vectorizerTFIDF_pres2G = TfidfVectorizer(ngram_range=(2,2))
vectorizerTFIDF_pres12G = TfidfVectorizer(ngram_range=(1,2))
vectorizerTFIDF_pres13G = TfidfVectorizer(ngram_range=(1,3))
vectorizerTFIDF_pres3G = TfidfVectorizer(ngram_range=(3,3))

Les combinaisons de paramètres maintenant créées, on peut s'attaquer à la réalisation du GridSearch.

In [145]:
# Code permettant de réaliser le GridSearch

# Création du dictionnaire afin de créer un DataFrame
df = dict()

# On ouvre le fichier contenant les configurations
with open("model_results.txt", 'w') as f:
    
    # On parcourt chaque ligne du fichier
    for it in range(len(paramsComb)) :
        print("Combinaison :", it + 1, "/", len(paramsComb), end='\r')
           
        # On récupère la configuration à utiliser
        params = paramsComb[it]
                
        # Utilisation de la fonction ut.transforme pour formater les données
        appX_t = ut.transform(appX, **params['processing'])
            
        # Initialisation du modèle
        m = params['model']['m']
        if (m == MultinomialNB) :
            model = MultinomialNB()
        elif (m == LinearSVC) :
            model = LinearSVC(C = params['cross_val']['C'])
        else :
            model = LogisticRegression(C = params['cross_val']['C'])
                    
        # Représentation sous forme vectorielle
        if (params['vectorizer']['type'] == CountVectorizer) :
            vectorizer = CountVectorizer()
        else :
            vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(appX_t)

        # Cross-validation
        kfold = KFold(n_splits=2, shuffle=True)
        scores = cross_validate(model, X, appY, cv=kfold, scoring=['accuracy','f1'])
        accuracy = np.mean(scores['test_accuracy'])
        f1 = np.mean(scores['test_f1'])
                    
        # Ecriture du score dans le fichier de sortie
        f.write('{} {} {}\n' . format(it, accuracy, f1))
                
        # Ecriture dans un dictionnaire pour l'affichage DataFrame
        line = params
        line['model']['m'] = str(line['model']['m'])
        line['vectorizer']['type'] = str(line['vectorizer']['type'])
                
        df[it] = [it] + [i for k, v in line.items() for i in v.values()]
        df[it].append(accuracy)

# On place le resultat dans un dataframe Pandas afin de faciliter son exploitation par la suite
res = pd.DataFrame.from_dict(df, orient='index', columns=['selected line', 'lowercase and no accents', 'no numbers', 'no punctuation', 'stemming', 'no stopwords', 'ngram', 'vectorizer', 'model', 'regularization factor', 'score'])

KeyboardInterrupt: 